In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterstats.io import bounds_window
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from treeinterpreter import treeinterpreter as ti

# Random Forest Model for Crop Type and Land Classification

Using data created by SERVIR East Africa, RCMRD, and FEWSET, we demonstrate how to train a Random Forest classifier over Trans Nzoia county, Kenya.

In [ ]:
%matplotlib inline

In [ ]:
# read in training data
training_vectors = gpd.read_file('training_data.geojson')
training_vectors.head()

In [ ]:
# show a single geometry
training_vectors.geometry[622]

In [ ]:
# find all unique values of training data names to use as classes
classes = np.unique(training_vectors.name)
classes

In [ ]:
# create a dictionary to convert class names into integers for modeling
class_dict = dict(zip(classes, range(len(classes))))
class_dict                  

In [ ]:
raster_file = 'https://agdic-servir-training.s3.amazonaws.com/sentinel2.tif'
with rasterio.open(raster_file, 'r') as src:
    # read in just the boundary of a single geometry
    geometry = training_vectors.geometry[62]
    window = bounds_window(geometry.bounds, src.transform)
    data = src.read(window=window)
    
    to_show = np.moveaxis(data[3:], 0, 2)
    plt.imshow(to_show)
    window_affine = src.window_transform(window)


In [ ]:
# this larger cell reads data from a raster file for each training vector
X_raw = []
y_raw = []

with rasterio.open(raster_file, 'r') as src:
    for (label, geom) in zip(training_vectors.name, training_vectors.geometry):
        # read the raster data matching the geometry bounds
        window = bounds_window(geom.bounds, src.transform)
        
        # store our window information
        window_affine = src.window_transform(window)
        fsrc = src.read(window=window)

        # rasterize the (non-buffered) geometry into the larger shape and affine
        mask = rasterize(
            [(geom, 1)],
            out_shape=fsrc.shape[1:],
            transform=window_affine,
            fill=0,
            dtype='uint8',
            all_touched=True
        ).astype(bool)

        # for each label pixel (places where the mask is true)...
        label_pixels = np.argwhere(mask)
        for (row, col) in label_pixels:
            # add a pixel of data to X
            data = fsrc[:,row,col]
            one_x = np.nan_to_num(data, nan=1e-3)
            X_raw.append(one_x)
            
            # add the label to y
            y_raw.append(class_dict[label])

In [ ]:
# convert the training data lists into the appropriate shape and format for scikit-learn
X = np.array(X_raw)
y = np.array(y_raw)
(X.shape, y.shape)

In [ ]:
# (optional) add extra band indices

# helper function for calculating ND*I indices (bands in the final dimension)
def band_index(arr, a, b):
    return np.expand_dims((arr[..., a] - arr[..., b]) / (arr[..., a] + arr[..., b]), axis=1)

ndvi = band_index(X, 3, 2)
ndwi = band_index(X, 1, 3)

X = np.concatenate([X, ndvi, ndwi], axis=1)
X.shape

In [ ]:
# split the data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# calculate class weights to allow for training on inbalanced training samples
labels, counts = np.unique(y_train, return_counts=True)
class_weight_dict = dict(zip(labels, 1 / counts))
class_weight_dict

In [ ]:
# initialize a RandomForestClassifier
clf = RandomForestClassifier(
    n_estimators=200,
    class_weight=class_weight_dict,
    max_depth=6,
    n_jobs=3,
    verbose=1,
    random_state=0
)

In [ ]:
# fit the model to the data (training)
clf.fit(X, y)

In [ ]:
# predict on X_test to evaluate the model
preds = clf.predict(X_test)
cm = confusion_matrix(y_test, preds, labels=labels)

In [ ]:
# plot the confusion matrix
%matplotlib inline
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
# We want to show all ticks...
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       # ... and label them with the respective list entries
       xticklabels=classes, yticklabels=classes,
       title='Normalized Confusion Matrix',
       ylabel='True label',
       xlabel='Predicted label')

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
fmt = '.2f'
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], fmt),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")
fig.tight_layout()

## Generate predictions over the full image

In [ ]:
# perform prediction on each small image patch to minimize required memory
patch_size = 500

# open connections to our input and output images
with rasterio.open(raster_file, 'r') as src:
    profile = src.profile
    profile.update(
        dtype=rasterio.uint8,
        count=1,
    )
    with rasterio.open(output_image, 'w', **profile) as dst:
        i = 3
        j = 3
    
        # define the pixels to read (and write)
        window = rasterio.windows.Window(
            j * patch_size,
            i * patch_size,
            # don't read past the image bounds
            min(patch_size, src.shape[1] - j * patch_size),
            min(patch_size, src.shape[0] - i * patch_size)
        )

        data = src.read(window=window)
        
        # read the image into the proper format, adding indices if necessary
        img_swp = np.moveaxis(data, 0, 2)
        img_flat = img_swp.reshape(-1, img_swp.shape[-1])

        img_ndvi = band_index(img_flat, 3, 2)
        img_ndwi = band_index(img_flat, 1, 3)

        img_w_ind = np.concatenate([img_flat, img_ndvi, img_ndwi], axis=1)
        to_predict = np.nan_to_num(img_w_ind)

        # predict
        img_preds = clf.predict(to_predict)
        
        # add the prediction back to the output
        # resize to the original image dimensions
        output = img_preds.flatten()
        output = output.reshape(*img_swp.shape[:-1])
        
        # visualize our result
        plt.imshow(output)

In [ ]:
with rasterio.open(raster_file, 'r') as src:

    i = 3
    j = 3

    # define the pixels to read (and write)
    window = rasterio.windows.Window(
        j * patch_size,
        i * patch_size,
        # don't read past the image bounds
        min(patch_size, src.shape[1] - j * patch_size),
        min(patch_size, src.shape[0] - i * patch_size)
    )

    data = src.read(window=window)
    plt.imshow(np.moveaxis(data[:3], 0, 2))